# CIFAR 10
Tanay Trivedi

In [0]:
import tensorflow as tf
import numpy as np

In [1]:
import keras

Using TensorFlow backend.


## Data

In [3]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
x_train=x_train.astype("float32")
y_train=y_train.astype("int32")
x_test=x_test.astype("float32")
y_test=y_test.astype("int32")


170508288/170498071 [==============================] - 96s 1us/step


Create validation set...

In [0]:
valSetSize = .1 
valIdx = int((1-valSetSize)*len(x_train))

x_val = x_train[valIdx:]
y_val = y_train[valIdx:]
x_train = x_train[:valIdx]
y_train = y_train[:valIdx]

## Base Configuration

In [0]:
def cnn_model_fn_1(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    conv1_bn = tf.layers.batch_normalization(pool1)

    conv2 = tf.layers.conv2d(
      inputs=conv1_bn,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    conv2_bn = tf.layers.batch_normalization(pool2)
    
    conv3 = tf.layers.conv2d(
      inputs=conv2_bn,
      filters=256,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    conv3_bn = tf.layers.batch_normalization(pool3)
    
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn,
      filters=512,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    conv4_bn = tf.layers.batch_normalization(pool4)

    pool4_flat = tf.contrib.layers.flatten(conv4_bn)  

    full1 = tf.layers.dense(inputs=pool4_flat, units=128, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    full1 = tf.layers.dense(inputs=full1, units=256, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    full1 = tf.layers.dense(inputs=full1, units=512, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    full1 = tf.layers.dense(inputs=full1, units=1024, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    
    
    logits = tf.layers.dense(inputs=full1, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    vs=tf.trainable_variables()
    loss += tf.add_n([tf.nn.l2_loss(v) for v in vs]) * 0.001
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
config=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn_1,config=config)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d6b2558d0>, '_model_dir': '/tmp/tmpLx292d', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


In [9]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=128,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=10000);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpLx292d/model.ckpt.
INFO:tensorflow:loss = 11.235537, step = 0
INFO:tensorflow:global_step/sec: 13.9994
INFO:tensorflow:loss = 4.734158, step = 1000 (71.433 sec)
INFO:tensorflow:global_step/sec: 14.0798
INFO:tensorflow:loss = 4.521372, step = 2000 (71.025 sec)
INFO:tensorflow:global_step/sec: 14.1106
INFO:tensorflow:loss = 4.559145, step = 3000 (70.871 sec)
INFO:tensorflow:global_step/sec: 14.0968
INFO:tensorflow:loss = 4.3638554, step = 4000 (70.935 sec)
INFO:tensorflow:global_step/sec: 14.1079
INFO:tensorflow:loss = 4.380804, step = 5000 (70.883 sec)
INFO:tensorflow:global_step/sec: 14.0931
INFO:tensorflow:loss = 4.2947364, step = 6000 (70.955 sec)
INFO:tensorflow:global_step/sec: 14.1145
INFO:tenso

### Validation

In [10]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-01-23:10:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpLx292d/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-01-23:10:46
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.3824, global_step = 10000, loss = 4.0393972
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmpLx292d/model.ckpt-10000
{'loss': 4.0393972, 'global_step': 10000, 'accuracy': 0.3824}


## Remove L2 Reg

In [0]:
def cnn_model_fn_2(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    conv1_bn = tf.layers.batch_normalization(pool1)

    conv2 = tf.layers.conv2d(
      inputs=conv1_bn,
      filters=128,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    conv2_bn = tf.layers.batch_normalization(pool2)
    
    conv3 = tf.layers.conv2d(
      inputs=conv2_bn,
      filters=256,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    conv3_bn = tf.layers.batch_normalization(pool3)
    
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn,
      filters=512,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    conv4_bn = tf.layers.batch_normalization(pool4)

    pool4_flat = tf.contrib.layers.flatten(conv4_bn)  

    full1 = tf.layers.dense(inputs=pool4_flat, units=128, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    full1 = tf.layers.dense(inputs=full1, units=256, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    full1 = tf.layers.dense(inputs=full1, units=512, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    full1 = tf.layers.dense(inputs=full1, units=1024, activation=tf.nn.relu)
    full1 = tf.layers.dropout(
      inputs=full1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    full1 = tf.layers.batch_normalization(full1)
    
    
    
    logits = tf.layers.dense(inputs=full1, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    #vs=tf.trainable_variables()
    #loss += tf.add_n([tf.nn.l2_loss(v) for v in vs]) * 0.001
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
config1=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier1 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_2,config=config1)

train_input_fn_1 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=128,
    num_epochs=None,
    shuffle=True)
mnist_classifier1.train(
    input_fn=train_input_fn_1,
    steps=10000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01d97f0890>, '_model_dir': '/tmp/tmpMVVUkV', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflo

### Validation

In [6]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier1.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-01-23:37:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpMVVUkV/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-01-23:37:58
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.543, global_step = 10000, loss = 1.3303111
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmpMVVUkV/model.ckpt-10000
{'loss': 1.3303111, 'global_step': 10000, 'accuracy': 0.543}


## Change Up Config

In [0]:
def cnn_model_fn_3(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    
    conv1_bn = tf.layers.batch_normalization(conv1)

    conv2 = tf.layers.conv2d(
      inputs=conv1_bn,
      filters=32,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv2_bn = tf.layers.batch_normalization(conv2)
    pool2 = tf.layers.max_pooling2d(inputs=conv2_bn, pool_size=[4, 4], strides=2)
    full1 = tf.layers.dropout(
      inputs=pool2, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    conv3 = tf.layers.conv2d(
      inputs=full1,
      filters=64,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    #pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    conv3_bn = tf.layers.batch_normalization(conv3)
    
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn,
      filters=64,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv4_bn = tf.layers.batch_normalization(conv4)
    pool4 = tf.layers.max_pooling2d(inputs=conv4_bn, pool_size=[4, 4], strides=2)
    full2 = tf.layers.dropout(
      inputs=pool4, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    pool4_flat = tf.contrib.layers.flatten(full2)  

    """full3 = tf.layers.dense(inputs=pool4_flat, units=256, activation=tf.nn.relu)
    full4 = tf.layers.dropout(
      inputs=full3, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)"""
    
    full5 = tf.layers.dense(inputs=pool4_flat, units=512, activation=tf.nn.relu)
    full6 = tf.layers.dropout(
      inputs=full5, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    logits = tf.layers.dense(inputs=full6, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [38]:
config2=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier2 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_3,config=config2)

train_input_fn_2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=128,
    num_epochs=None,
    shuffle=True)
mnist_classifier2.train(
    input_fn=train_input_fn_2,
    steps=20000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c77148a90>, '_model_dir': '/tmp/tmpX5qHeS', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

### Validation

In [39]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier2.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-03-03:29:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpX5qHeS/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-03-03:29:54
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.7556, global_step = 20000, loss = 0.8347336
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpX5qHeS/model.ckpt-20000
{'loss': 0.8347336, 'global_step': 20000, 'accuracy': 0.7556}


## More Conv!

In [0]:
def cnn_model_fn_4(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    
    conv1_bn = tf.layers.batch_normalization(conv1)

    conv2 = tf.layers.conv2d(
      inputs=conv1_bn,
      filters=32,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv2_bn = tf.layers.batch_normalization(conv2)
    pool2 = tf.layers.max_pooling2d(inputs=conv2_bn, pool_size=[4, 4], strides=2)
    full1 = tf.layers.dropout(
      inputs=pool2, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    conv3 = tf.layers.conv2d(
      inputs=full1,
      filters=64,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    #pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    conv3_bn = tf.layers.batch_normalization(conv3)
    
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn,
      filters=64,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv4_bn = tf.layers.batch_normalization(conv4)
    pool4 = tf.layers.max_pooling2d(inputs=conv4_bn, pool_size=[4, 4], strides=2)
    full2 = tf.layers.dropout(
      inputs=pool4, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    conv3 = tf.layers.conv2d(
      inputs=full2,
      filters=128,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    #pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    conv3_bn = tf.layers.batch_normalization(conv3)
    
    
    conv5 = tf.layers.conv2d(
      inputs=conv3_bn,
      filters=128,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv5_bn = tf.layers.batch_normalization(conv5)
    pool5 = tf.layers.max_pooling2d(inputs=conv5_bn, pool_size=[4, 4], strides=2)
    full7 = tf.layers.dropout(
      inputs=pool5, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    pool4_flat = tf.contrib.layers.flatten(full7)  

    """full3 = tf.layers.dense(inputs=pool4_flat, units=256, activation=tf.nn.relu)
    full4 = tf.layers.dropout(
      inputs=full3, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)"""
    
    full5 = tf.layers.dense(inputs=pool4_flat, units=512, activation=tf.nn.relu)
    full6 = tf.layers.dropout(
      inputs=full5, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    logits = tf.layers.dense(inputs=full6, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [41]:
config2=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier2 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_4,config=config2)

train_input_fn_2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=128,
    num_epochs=None,
    shuffle=True)
mnist_classifier2.train(
    input_fn=train_input_fn_2,
    steps=20000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c7b094790>, '_model_dir': '/tmp/tmpQY9IBZ', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

### Validation

In [42]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier2.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-03-03:53:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpQY9IBZ/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-03-03:53:02
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.7628, global_step = 20000, loss = 1.0546504
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpQY9IBZ/model.ckpt-20000
{'loss': 1.0546504, 'global_step': 20000, 'accuracy': 0.7628}


## More Dense!

In [0]:
def cnn_model_fn_5(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    
    conv1_bn = tf.layers.batch_normalization(conv1)

    conv2 = tf.layers.conv2d(
      inputs=conv1_bn,
      filters=32,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv2_bn = tf.layers.batch_normalization(conv2)
    pool2 = tf.layers.max_pooling2d(inputs=conv2_bn, pool_size=[4, 4], strides=2)
    full1 = tf.layers.dropout(
      inputs=pool2, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    conv3 = tf.layers.conv2d(
      inputs=full1,
      filters=64,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    #pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    conv3_bn = tf.layers.batch_normalization(conv3)
    
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn,
      filters=64,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv4_bn = tf.layers.batch_normalization(conv4)
    pool4 = tf.layers.max_pooling2d(inputs=conv4_bn, pool_size=[4, 4], strides=2)
    full2 = tf.layers.dropout(
      inputs=pool4, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    conv3 = tf.layers.conv2d(
      inputs=full2,
      filters=128,
      kernel_size=[8, 8],
      padding="same",
      activation=tf.nn.relu)
    #pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    conv3_bn = tf.layers.batch_normalization(conv3)
    
    
    conv5 = tf.layers.conv2d(
      inputs=conv3_bn,
      filters=128,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    #
    conv5_bn = tf.layers.batch_normalization(conv5)
    pool5 = tf.layers.max_pooling2d(inputs=conv5_bn, pool_size=[4, 4], strides=2)
    full7 = tf.layers.dropout(
      inputs=pool5, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    pool4_flat = tf.contrib.layers.flatten(full7)  

    full3 = tf.layers.dense(inputs=pool4_flat, units=256, activation=tf.nn.relu)
    full4 = tf.layers.dropout(
      inputs=full3, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    full5 = tf.layers.dense(inputs=full4, units=512, activation=tf.nn.relu)
    full6 = tf.layers.dropout(
      inputs=full5, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    
    logits = tf.layers.dense(inputs=full6, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [44]:
config2=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier2 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_5,config=config2)

train_input_fn_2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=128,
    num_epochs=None,
    shuffle=True)
mnist_classifier2.train(
    input_fn=train_input_fn_2,
    steps=20000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c6d237590>, '_model_dir': '/tmp/tmpqc_O5R', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

### Validation

In [45]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier2.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-03-04:16:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqc_O5R/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-03-04:16:56
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.7668, global_step = 20000, loss = 0.94575536
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpqc_O5R/model.ckpt-20000
{'loss': 0.94575536, 'global_step': 20000, 'accuracy': 0.7668}


## Keras Version! w/ Learning Rate Decay

In [0]:
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch):          
    return 0.001*np.exp(-0.02*epoch)
lr=LearningRateScheduler(lr_schedule,verbose=0)
#inspired by Jacob Maareks implementation, but i customized a smooth function rather than his step function

In [0]:
model=keras.models.Sequential()
#conv layer 1
model.add(keras.layers.Conv2D(32,(8,8),padding="same",input_shape=x_train.shape[1:]))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(4,4),strides=2))
model.add(keras.layers.Dropout(0.25))

#conv layer 2
model.add(keras.layers.Conv2D(64,(8,8),padding="same"))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(4,4),strides=2))
model.add(keras.layers.Dropout(0.25))

#dense layer
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10)) # 10 labels
model.add(keras.layers.Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])




In [0]:
class NBatchLogger(keras.callbacks.Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_batch_end(self, batch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log = ''
            for (k, v) in self.metric_cache.items():
                val = v / self.display
                if abs(val) > 1e-3:
                    metrics_log += ' - %s: %.4f' % (k, val)
                else:
                    metrics_log += ' - %s: %.4e' % (k, val)
            print('step: {}/{} ... {}'.format(self.step,
                                          self.params['steps'],
                                          metrics_log))
            self.metric_cache.clear()

In [0]:
y_train_k=keras.utils.np_utils.to_categorical(y_train)
y_val_k=keras.utils.np_utils.to_categorical(y_val)
y_test_k=keras.utils.np_utils.to_categorical(y_test)

In [89]:
out_batch = NBatchLogger(display=10000)
model.fit(x=x_train, y=y_train_k, batch_size=100, epochs=75, verbose=0, callbacks=[out_batch,lr], validation_data=(x_val,y_val_k), shuffle=True)

step: 10000/None ...  - acc: 0.7450 - loss: 0.7325
step: 20000/None ...  - acc: 0.9039 - loss: 0.2730
step: 30000/None ...  - acc: 0.9457 - loss: 0.1553


### Validation 75epochs

In [0]:
PREDICTED_CLASSES = model.predict_classes(x_val, verbose=0)


In [91]:
temp = sum(y_val[:,0] == PREDICTED_CLASSES)
temp/float(len(y_val[:,0]))

0.8148

### Validation 100 epochs

In [0]:
PREDICTED_CLASSES = model.predict_classes(x_val, verbose=0)


In [82]:
temp = sum(y_val[:,0] == PREDICTED_CLASSES)
temp/float(len(y_val[:,0]))

0.8238

### Validation 125 epochs

In [0]:
PREDICTED_CLASSES = model.predict_classes(x_val, verbose=0)


In [86]:
temp = sum(y_val[:,0] == PREDICTED_CLASSES)
temp/float(len(y_val[:,0]))

0.8044

## Keras more dense


In [0]:
model=keras.models.Sequential()
#conv layer 1
model.add(keras.layers.Conv2D(32,(8,8),padding="same",input_shape=x_train.shape[1:]))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(4,4),strides=2))
model.add(keras.layers.Dropout(0.25))

#conv layer 2
model.add(keras.layers.Conv2D(64,(8,8),padding="same"))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(4,4),strides=2))
model.add(keras.layers.Dropout(0.25))

#conv layer 3
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1024))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10))
model.add(keras.layers.Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])




In [29]:
out_batch = NBatchLogger(display=10000)
model.fit(x=x_train, y=y_train_k, batch_size=100, epochs=100, verbose=0, callbacks=[out_batch], validation_data=(x_val,y_val_k), shuffle=True)

step: 10000/None ...  - acc: 0.7255 - loss: 0.7977
step: 20000/None ...  - acc: 0.8746 - loss: 0.3667
step: 30000/None ...  - acc: 0.9178 - loss: 0.2414
step: 40000/None ...  - acc: 0.9381 - loss: 0.1842


### Validation

In [0]:
PREDICTED_CLASSES = model.predict_classes(x_val, verbose=0)


In [31]:
temp = sum(y_val[:,0] == PREDICTED_CLASSES)
temp/float(len(y_val[:,0]))

0.8112

## Keras deep and lr


In [0]:
from keras.optimizers import Adam

In [0]:
model=keras.models.Sequential()
#conv layer 1
model.add(keras.layers.Conv2D(64,(8,8),padding="same",input_shape=x_train.shape[1:]))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(4,4),strides=2))
model.add(keras.layers.Dropout(0.25))

#conv layer 2
model.add(keras.layers.Conv2D(128,(8,8),padding="same"))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(8,8),strides=2))
model.add(keras.layers.Dropout(0.25))

#conv layer 3
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10))
model.add(keras.layers.Activation("softmax"))

opt=Adam()
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])




In [63]:
out_batch = NBatchLogger(display=1000)
model.fit(x=x_train, y=y_train_k, batch_size=100, epochs=100, verbose=0, callbacks=[out_batch,lr], validation_data=(x_val,y_val_k), shuffle=True)

step: 24000/None ...  - acc: 0.9658 - loss: 0.1748
step: 25000/None ...  - acc: 0.9659 - loss: 0.1847
step: 26000/None ...  - acc: 0.9691 - loss: 0.1661
step: 27000/None ...  - acc: 0.9717 - loss: 0.1578
step: 28000/None ...  - acc: 0.9738 - loss: 0.1489
step: 29000/None ...  - acc: 0.9735 - loss: 0.1530
step: 30000/None ...  - acc: 0.9763 - loss: 0.1394
step: 31000/None ...  - acc: 0.9784 - loss: 0.1358
step: 32000/None ...  - acc: 0.9776 - loss: 0.1413
step: 33000/None ...  - acc: 0.9755 - loss: 0.1465
step: 34000/None ...  - acc: 0.9780 - loss: 0.1383
step: 35000/None ...  - acc: 0.9806 - loss: 0.1288
step: 36000/None ...  - acc: 0.9815 - loss: 0.1258
step: 37000/None ...  - acc: 0.9832 - loss: 0.1159
step: 38000/None ...  - acc: 0.9803 - loss: 0.1260
step: 39000/None ...  - acc: 0.9801 - loss: 0.1283
step: 40000/None ...  - acc: 0.9831 - loss: 0.1170
step: 41000/None ...  - acc: 0.9827 - loss: 0.1203
step: 42000/None ...  - acc: 0.9857 - loss: 0.1067
step: 43000/None ...  - acc: 0.

### Validation

In [0]:
PREDICTED_CLASSES = model.predict_classes(x_val, verbose=0)


In [65]:
temp = sum(y_val[:,0] == PREDICTED_CLASSES)
temp/float(len(y_val[:,0]))

0.817

## Test

In [74]:
PREDICTED_CLASSES = model.predict_classes(x_test, verbose=0)
temp = sum(y_test[:,0] == PREDICTED_CLASSES)
temp/float(len(y_test[:,0]))

0.8066

# CIFAR 100

In [93]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar100.load_data()
x_train=x_train.astype("float32")
y_train=y_train.astype("int32")
x_test=x_test.astype("float32")
y_test=y_test.astype("int32")

valSetSize = .1 
valIdx = int((1-valSetSize)*len(x_train))

x_val = x_train[valIdx:]
y_val = y_train[valIdx:]
x_train = x_train[:valIdx]
y_train = y_train[:valIdx]

y_train_k=keras.utils.np_utils.to_categorical(y_train)
y_val_k=keras.utils.np_utils.to_categorical(y_val)
y_test_k=keras.utils.np_utils.to_categorical(y_test)

169017344/169001437 [==============================] - 50s 0us/step


## Same Model

In [0]:
model=keras.models.Sequential()
#conv layer 1
model.add(keras.layers.Conv2D(32,(8,8),padding="same",input_shape=x_train.shape[1:]))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(4,4),strides=2))
model.add(keras.layers.Dropout(0.25))

#conv layer 2
model.add(keras.layers.Conv2D(64,(8,8),padding="same"))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64,(4,4)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(4,4),strides=2))
model.add(keras.layers.Dropout(0.25))

#conv layer 3
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(100)) #fine label
model.add(keras.layers.Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy","top_k_categorical_accuracy"])




In [97]:
out_batch = NBatchLogger(display=1000)
model.fit(x=x_train, y=y_train_k, batch_size=100, epochs=100, verbose=0, callbacks=[out_batch], validation_data=(x_val,y_val_k), shuffle=True)

step: 1000/None ...  - acc: 0.1122 - loss: 3.8975 - top_k_categorical_accuracy: 0.3169
step: 2000/None ...  - acc: 0.2289 - loss: 3.1407 - top_k_categorical_accuracy: 0.5212
step: 3000/None ...  - acc: 0.2994 - loss: 2.7726 - top_k_categorical_accuracy: 0.6144
step: 4000/None ...  - acc: 0.3530 - loss: 2.5048 - top_k_categorical_accuracy: 0.6727
step: 5000/None ...  - acc: 0.3919 - loss: 2.3194 - top_k_categorical_accuracy: 0.7119
step: 6000/None ...  - acc: 0.4213 - loss: 2.1872 - top_k_categorical_accuracy: 0.7407
step: 7000/None ...  - acc: 0.4498 - loss: 2.0474 - top_k_categorical_accuracy: 0.7666
step: 8000/None ...  - acc: 0.4735 - loss: 1.9341 - top_k_categorical_accuracy: 0.7903
step: 9000/None ...  - acc: 0.4948 - loss: 1.8367 - top_k_categorical_accuracy: 0.8076
step: 10000/None ...  - acc: 0.5110 - loss: 1.7747 - top_k_categorical_accuracy: 0.8191
step: 11000/None ...  - acc: 0.5321 - loss: 1.6789 - top_k_categorical_accuracy: 0.8373
step: 12000/None ...  - acc: 0.5443 - los

In [100]:
model.evaluate(x_test,y_test_k)

5000/5000 [==============================] - 1s 294us/step


[2.0736297103881838, 0.5116, 0.7886]

Top 5 accuracy is 78%

# Conclusion and Customization

More or less my customization moves have been to increase or decrease convolutional filter numbers or size, use pooling and batch normalization either frequently or infrequently and increase the size and width of dense layers.

Another step i took was to convert to keras, for some reason this increased performance even though I cannot figure out the difference between the two. As far as I can tell the graphs are the same. I think Keras has some kind of early stopping on bad training cycles that is not default in TensorFlow.

It is questionable if the learning rate decay I used is useful. I was hinted onto it from Jacob Maarek, and he gave me some scale for the decay which i built into my smooth function. I didn't have enough time to explore all the parameters, so I think I could boost this with a simpler optimizer than Adams so I would have more control over the decay.